# 🗺️Getting our Bearings🗺️
## Goal: Look at images of the same type and predict the labels of each individual cell within those images.
* Task 1: Develop segmentation to cut images into images containing single-cells.
* Task 2: Develop and train a CNN to predict labels (using green channel only) of single-cells
* Task 3: Predict labels for all cells in "test" images - This is our submission.

In [ ]:
!ls ../input/hpa-single-cell-image-classification

In [ ]:
#Imports

#Much of the code used here is based on: https://keras.io/examples/keras_recipes/tfrecord/

import pandas as pd
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid

import glob

from PIL import Image

from keras.models import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

from sklearn.model_selection import train_test_split
from functools import partial

label_map = {'0': 'Nucleoplasm',
             '1': 'Nuclear membrane',
             '2': 'Nucleoli',
             '3': 'Nucleoli fibrillar center',
             '4': 'Nuclear speckles',
             '5': 'Nuclear bodies',
             '6': 'Endoplasmic reticulum',
             '7': 'Golgi apparatus',
             '8': 'Intermediate filaments',
             '9': 'Actin filaments',
             '10': 'Microtubules',
             '11': 'Mitotic spindle',
             '12': 'Centrosome',
             '13': 'Plasma membrane',
             '14': 'Mitochondria',
             '15': 'Aggresome',
             '16': 'Cytosol',
             '17': 'Vesicles and punctate cytosolic patterns',
             '18': 'Negative'
            }

sample_submit = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
sample_submit.head(3)

In [ ]:
train_csv = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
train_csv.head(3)

In [ ]:
# Spicy...we not only have balance issues, we have multiple cell types per image
train_csv['Label'].value_counts()

In [ ]:
#Filtering down to a list of images with single cell types:
train_csv[train_csv['Label'].isin(label_map.keys())]['Label'].value_counts()

# 🔬 Image Exploration 🔬
The first real challenge is going to be unsupervised segmentation so let's see what we're up against.

In [ ]:
train_dir = '/kaggle/input/hpa-single-cell-image-classification/train_tfrecords/*'
test_dir = '/kaggle/input/hpa-single-cell-image-classification/test_tfrecords/*'
train_list = glob.glob(train_dir)
test_list = glob.glob(test_dir)

print("Train Files: ", len(train_list))
print("Test Files: ", len(test_list))

In [ ]:
# A utility script for reading features out of TFRecords:
# I think I found it on stack exchange a while ago...

def list_record_features(tfrecords_path):
    # Dict of extracted feature information
    features = {}
    # Iterate records
    for rec in tf.data.TFRecordDataset([str(tfrecords_path)]):
        # Get record bytes
        example_bytes = rec.numpy()
        # Parse example protobuf message
        example = tf.train.Example()
        example.ParseFromString(example_bytes)
        # Iterate example features
        for key, value in example.features.feature.items():
            # Kind of data in the feature
            kind = value.WhichOneof('kind')
            # Size of data in the feature
            size = len(getattr(value, kind).value)
            # Check if feature was seen before
            if key in features:
                # Check if values match, use None otherwise
                kind2, size2 = features[key]
                if kind != kind2:
                    kind = None
                if size != size2:
                    size = None
            # Save feature data
            features[key] = (kind, size)
    return features

record_features = list_record_features(glob.glob(train_dir+"*")[0])
print(record_features)

In [ ]:
# We start with my basic data generator for unpacking tfrecords.
# This will get tuned in for the dataset in question.

def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    #image = tf.image.resize_with_crop_or_pad(image, 512, 512)
    image = tf.image.resize(image, size=[im_size[0], im_size[1]])
    return image


def read_tfrecord(example, labeled=True):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "target": tf.io.FixedLenFeature([], tf.string),
            "image_name": tf.io.FixedLenFeature([], tf.string)
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        #label = tf.one_hot(example["target"], depth = 19)
        label = example['target']
        #return example
        name = example["image_name"]
        return image, label, name
    else:
        #return example
        return image


def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset


def get_dataset(filenames, batch_size=10, labeled=True):
    image_size = im_size
    dataset = load_dataset(filenames, labeled)
    # Normally I would shuffle the dataset here but we need our channels not to get jumbled (for now :p )
    #dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    return dataset

In [ ]:
im_size = [200,200,4]
batch_size = 25

train_dataset = get_dataset(train_list, batch_size=batch_size, labeled=True)
valid_dataset = get_dataset(test_list, batch_size=batch_size, labeled=True)
image_batch, label_batch, name_batch = next(iter(train_dataset))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        ax.title.set_text(str(label_batch[n].numpy()).replace("b", "").replace("'", ""))
        plt.imshow(image_batch[n] / 255.0)
        plt.axis("off")
        
show_batch(image_batch, name_batch)

In [ ]:
batch_size = 1000
train_dataset = get_dataset(train_list, batch_size=batch_size, labeled=True)
image_batch, label_batch, name_batch = next(iter(train_dataset))
name_batch

# To produce composite images we run into something of a problem.
# Our data-generator produces info out of order.
# We would need to take the batch, match the images with their mates and provide the model with that stack for segmentation.
# That's where I'll leave off 2/23

## Masking
So we found we have an R, G, B and Y channel reflecting tagged proteins in each image. Let's see if we can get the HPA-Cell-Segmentator working.

In [ ]:
!pip install 'git+https://github.com/haoxusci/pytorch_zoo@master#egg=pytorch_zoo'
!pip install 'https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip'

In [ ]:
image_batch[0].numpy()

In [ ]:
# Example useage from the HPA github:

NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=False,
    multi_channel_model=True,
)

# Passing the numpy arrays of the images as they are (float32) causes: OverflowError: cannot convert float infinity to integer
# TODO: Fix above error and replace this comment with the solution.

# Testing:
# From SkyDevour - takes composite image and returns image masks:
def get_masks(imgs, test=True):
    try:
        images = [[img[:, :, 0] for img in imgs], 
                  [img[:, :, 3] for img in imgs], 
                  [img[:, :, 2] for img in imgs],]
    
        nuc_segmentations = segmentator.pred_nuclei(images[2])
        cell_segmentations = segmentator.pred_cells(images)
        cell_masks = []
        for i in tqdm(range(len(cell_segmentations)), desc='Labeling cells..'):
            _, cell_mask = label_cell(nuc_segmentations[i], cell_segmentations[i])
            cell_masks.append(cell_mask)
        return cell_masks
    except:
        raise ValueError('Segmentation failed')

In [ ]:
composite = np.concatenate((image_batch[1].numpy(),image_batch[0].numpy(), image_batch[3].numpy()), axis = 2)

In [ ]:
plt.imshow(composite)

In [ ]:
def vis_integrated_gradients_masks_test(img_idx, conf_threshold=0.01, mask_height=2048, mask_width=2048, 
                                        max_cell_level_conf_2_image_level_conf=0.005, test_ids=test_ids,
                                        model=model_rgby, quantile_level=0.9, figsize=7):
    image_id = test_ids[img_idx]
    img = [cv2.resize(cv2.imread(os.path.join(TEST_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE),
                      (mask_height, mask_width))
           for color in ['red','green','blue','yellow']]
    img = np.stack(img, axis=-1)
    mask = get_masks([img])[0]
    n_cells = mask.max()
    cell_2_max_conf = dict()   
    
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH)).astype(np.float32)/255.
    predictions_test = model.predict(np.expand_dims(img, 0))
    
    for class_i, class_name in enumerate(class_names[:-1]):
        class_conf_score = predictions_test[0][class_i]
        if class_conf_score > conf_threshold:
            try:
                explanation = explainer.explain(([img], None), model, class_i, n_steps=15)
                explanation_img = cv2.resize(explanation, (mask_height, mask_width))
                explanation_total_level = np.quantile(explanation_img.flatten(), quantile_level)
            except:
                continue

            plt.figure(figsize=(figsize, figsize))
            plt.imshow(mask)
            plt.imshow(explanation_img, alpha=0.7)
            plt.xticks([])
            plt.yticks([])
            plt.title(f'{test_ids[img_idx]}\n{class_name} ({class_conf_score:.2f}): raw Grad-CAMs', fontsize=22)
            plt.show()

            masks_all = np.zeros((mask_height, mask_width))
            coord_2_conf = dict()
            for cell_i in range(1, n_cells + 1):
                cell_mask_bool = mask == cell_i
                cell_explanation_perc = np.quantile(explanation_img[cell_mask_bool], quantile_level)
                cell_conf = np.clip(cell_explanation_perc*class_conf_score/explanation_total_level, 0, class_conf_score)
                if cell_conf/class_conf_score >= max_cell_level_conf_2_image_level_conf and cell_conf > 1e-3:
                    masks_all[cell_mask_bool] = 1
                    mask_pixels_x, mask_pixels_y = np.where(cell_mask_bool)
                    coord_2_conf[(int(mask_pixels_y.mean()), int(mask_pixels_x.mean()))] = cell_conf
                    if not cell_i in cell_2_max_conf:
                        cell_2_max_conf[cell_i] = cell_conf
                    else:
                        cell_2_max_conf[cell_i] = max(cell_conf, cell_2_max_conf[cell_i])

            plt.figure(figsize=(figsize, figsize)) 
            plt.imshow(masks_all)
            for coords, conf in coord_2_conf.items():
                conf_rounded = np.round(conf*100)/100
                plt.scatter(*coords, s=700, color='red', marker=r"$ {} $".format(conf_rounded))
            plt.xticks([])
            plt.yticks([])
            plt.title(f'{test_ids[img_idx]}\n{class_name}: cell-level predictions', fontsize=22)
            plt.show()

    masks_all = np.zeros((mask_height, mask_width))
    coord_2_conf = dict()
    for cell_i in range(1, n_cells + 1):
        if not cell_i in cell_2_max_conf:
            cell_conf = 0.99
        else:
            cell_conf = 1 - cell_2_max_conf[cell_i]
        if cell_conf >= conf_threshold:
            cell_mask_bool = mask == cell_i
            masks_all[cell_mask_bool] = 1
            mask_pixels_x, mask_pixels_y = np.where(cell_mask_bool)
            coord_2_conf[(int(mask_pixels_y.mean()), int(mask_pixels_x.mean()))] = cell_conf

    plt.figure(figsize=(9, 9)) 
    plt.imshow(masks_all)
    for coords, conf in coord_2_conf.items():
        conf_rounded = np.round(conf*100)/100
        plt.scatter(*coords, s=700, color='red', marker=r"$ {} $".format(conf_rounded))
    plt.xticks([])
    plt.yticks([])
    plt.title(f'{test_ids[img_idx]}\n{class_names[-1]}: cell-level predictions', fontsize=22)
    plt.show()
    


In [ ]:
type(np.asarray(images[CHANNEL_BLUE])[0][0])

In [ ]:
plt.imshow(segmentator.pred_nuclei([(np.asarray(images[CHANNEL_BLUE]))])[0])

In [ ]:
np.asarray(images[CHANNEL_BLUE])

In [ ]:
image = image_batch[0].numpy()
image = tf.cast(image, tf.float32)
image = image.numpy()
image
#segmentator.pred_nuclei(image)

In [ ]:
nuclei_mask = label_nuclei(nuc_segmentations[0])
print(np.shape(nuclei_mask))

cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
print(np.shape(nuclei_mask))
print(np.shape(cell_mask))

fig = plt.figure(figsize=(25,25))

# Nuclei mask
nuclei_image = Image.fromarray( np.uint8(nuclei_mask) )

ax = fig.add_subplot(1, 4, 1)
ax.set_title("Nuclei Mask")
plt.imshow(np.asarray(nuclei_image))

# Cell nuclei mask
cell_nuclei_image = Image.fromarray( np.uint8(cell_nuclei_mask) )

ax = fig.add_subplot(1, 4, 2)
ax.set_title("Cell Nuclei Mask")
plt.imshow(np.asarray(cell_nuclei_image))

# Cell mask
cell_image = Image.fromarray( np.uint8(cell_mask) )

ax = fig.add_subplot(1, 4, 3)
ax.set_title("Cell Mask")
plt.imshow(np.asarray(cell_image))

In [ ]:
# Get unique vector of segment numbers
numbers = set( np.ravel(cell_mask) )
numbers.remove(0)

fig = plt.figure(figsize=(25,6*len(numbers)/4))
index = 1

# plot original cell mask from above
ax = fig.add_subplot((len(numbers)//4)+1, 4, index)
ax.set_title("Complete Cell Mask")
plt.imshow(np.asarray(cell_image))
index = index + 1

for number in numbers:
    # set all other 'numbers' to zero in cell mask
    isolated = np.where(cell_mask == number, cell_mask, 0)

    # plot isolated image
    ax = fig.add_subplot((len(numbers)//4)+1, 4, index)
    ax.set_title("Segment: {}".format(number))

    plt.imshow(isolated)
    index = index + 1

# Classification Model
Now that we have a segmentation strategy we can bake it into our data-generator.

In [ ]:
# So let's tweak our image decoder slightly.
# We want to segment cells, then only pass the green channel back since that's what we're using to predict classes.

def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, size=[im_size[0], im_size[1]])
    return image

In [ ]:
#So now our generator is giving us:

im_size = [200,200,4]
batch_size = 25

train_dataset = get_dataset(train_list, batch_size=batch_size, labeled=True)
valid_dataset = get_dataset(test_list, batch_size=batch_size, labeled=True)
image_batch, label_batch = next(iter(train_dataset))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n + 1)
        ax.set_title(label_batch[n])
        plt.imshow(image_batch[n] / 255.0)
        plt.axis("off")
        
show_batch(image_batch, label_batch)

In [ ]:
#Global variables for training
BATCH_SIZE = 25
im_size = [299, 299, 3]
keras_batch_size = BATCH_SIZE
learning_rate = 0.095
epochs = 1

In [ ]:
sample_train = next(iter(train_dataset))

In [ ]:
str_format = str(sample_train[1][4].numpy())
str_format.r

So far we have a dataset of images paired to lists of correct outputs contained in strings with format b'x|y|z...'

In [ ]:
str(sample_train[1][0].numpy())

In [ ]:
model = tf.keras.applications.InceptionV3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1,
    classifier_activation="softmax",
)

model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy,
              metrics=tf.keras.metrics.categorical_accuracy)

model.fit(train_dataset, validation_data=valid_dataset, batch_size=keras_batch_size, epochs=epochs)